In [3]:
!pip install peft

     |████████████████████████████████| 72 kB 1.7 MB/s             
     |████████████████████████████████| 619.9 MB 6.9 kB/s              
     |████████████████████████████████| 1.3 MB 57.0 MB/s            
     |████████████████████████████████| 63.3 MB 112 kB/s             
     |████████████████████████████████| 21.0 MB 51.6 MB/s            
     |████████████████████████████████| 177.1 MB 54 kB/s              
     |████████████████████████████████| 173.2 MB 20 kB/s              
     |████████████████████████████████| 11.8 MB 62.9 MB/s            
     |████████████████████████████████| 557.1 MB 3.5 kB/s              
     |████████████████████████████████| 168.4 MB 52 kB/s               
     |████████████████████████████████| 54.6 MB 201 kB/s             
     |████████████████████████████████| 317.1 MB 15 kB/s               
     |████████████████████████████████| 5.7 MB 75.7 MB/s            
     |████████████████████████████████| 849 kB 71.0 MB/s            
     |█████████

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [7]:
model_name = "EleutherAI/polyglot-ko-1.3b"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 │   task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout    │
│    5 )                                                                                           │
│    6                                                                                             │
│ ❱  7 model = AutoModelForCausalLM.from_pretrained(model_name)                                    │
│    8 model = get_peft_model(model, peft_config)                                                  │
│    9                                                                                             │
│   10 model.print_trainable_parameters()                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471 in           │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   468 │   │   │   )                                                                              │
│   469 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   470 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 471 │   │   │   return model_class.from_pretrained(                                            │
│   472 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   473 │   │   │   )                                                                              │
│   474 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/transformers/modeling_utils.py:2795 in from_pretrained    │
│                                                                                                  │
│   2792 │   │   │   │   mismatched_keys,                                                          │
│   2793 │   │   │   │   offload_index,                                                            │
│   2794 │   │   │   │   error_msgs,                                                               │
│ ❱ 2795 │   │   │   ) = cls._load_pretrained_model(                                               │
│   2796 │   │   │   │   model,                                                                    │
│   2797 │   │   │   │   state_dict,                                                               │
│   2798 │   │   │   │   loaded_state_dict_keys,  # XXX: rename?                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/transformers/modeling_utils.py:3109 in                    │
│ _load_pretrained_model                                                                           │
│                                                                                                  │
│   3106 │   │   │   │   # Skip the load for shards that only contain disk-offloaded weights when  │
│   3107 │   │   │   │   if shard_file in disk_only_shard_files:                                   │
│   3108 │   │   │   │   │   continue                                                              │
│ ❱ 3109 │   │   │   │   state_dict = load_state_dict(shard_file)                                  │
│   3110 │   │   │   │                                       

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right", model_max_length=512,
)